# Data loading and preprocessing

In this notebook we retrieve our dataset and preprocess it into a format that is ready to use for training our matrix factorization based recommender system.

First, we import the necessary Python packages.

In [ ]:
import gdown
import numpy as np
import pandas as pd
import scipy.sparse
from tqdm.auto import tqdm
import itertools
import json
import gzip
import math
tqdm.pandas() # for progress_apply etc.

## Loading

Next, we download the files for our dataset (Goodreads). We use the gdown package to retrieve them from the Google Drive they're originally hosted on. 

> Since we will be implementing a collaborative filtering algorithm, we only need the interactions part of the dataset. The code for reading in the other parts of the dataset were left as comments for potential future reference.

We define the URLs for each file...

In [ ]:
URLS = {
    # "BOOKS": "https://drive.google.com/uc?id=1ICk5x0HXvXDp5Zt54CKPh5qz1HyUIn9m",
    # "AUTHORS": "https://drive.google.com/uc?id=19cdwyXwfXx_HDIgxXaHzH0mrx8nMyLvC",
    # "REVIEWS": "https://drive.google.com/u/0/uc?id=1V4MLeoEiPQdocCbUHjR_7L9ZmxTufPFe",
    "INTERACTIONS": "https://drive.google.com/uc?id=1CCj-cQw_mJLMdvF_YYfQ7ibKA-dC_GA2"
}

and download each file. (if they haven't been downloaded in a previous run of the notebook)

In [ ]:
for name, url in URLS.items():
    gdown.cached_download(url, f"./data/{name}.json.gz", quiet=False)

We now define a function to read the dataset into a Pandas dataframe. This implementation is faster and more memory efficient than the read_json function provided by Pandas.

In [ ]:
from collections import defaultdict

def read_json_fast(filename, nrows=None):
  """
  Loads line delimited JSON files faster than the 
  read_json function provided by Pandas.
  
  Iterates over file line per line, so shouldn't
  cause out-of-memory issues, except if resulting
  DataFrame is too big.
  
  Args:
    filename: path of the JSON file
    nrows: total number of rows to read from the file
  
  Returns:
    Pandas DataFrame containing (part of) the data. 
  """
  with gzip.open(filename) as f:
        d = defaultdict(list)
        
        print(f"Processing {filename.split('/')[-1]}:")
        if nrows is not None:
            pbar = tqdm(itertools.islice(f, nrows), unit="lines")
        else:
            pbar = tqdm(f, unit="lines")
        for l in pbar:
            for k,v in json.loads(l).items():
                d[k].append(v) 
        return pd.DataFrame(d)

We define the dataset file locations...

In [ ]:
# books_file = './data/BOOKS.json.gz' # book metadata
interactions_file = './data/INTERACTIONS.json.gz' # user-book interactions (ratings)
# reviews_file = './data/REVIEWS.json.gz' # user-book interactions (reviews)
# authors_file = './data/AUTHORS.json.gz' # author metadata

and load the necessary files.

In [ ]:
%%time
# df_books = read_json_fast(books_file)
df_interactions = read_json_fast(interactions_file)
# df_authors =  read_json_fast(authors_file)

Now we look at the contents of the loaded dataset.

In [ ]:
display(df_interactions.head(1))

We can see that the dataset contains quite a few columns that are of no use to us. To make everything a little less cluttered we remove the columns that we don't use from the dataframe.

In [ ]:
df_interactions = df_interactions[['user_id', 'book_id', 'rating', 'date_updated']]

In [ ]:
display(df_interactions.head(1))

## Pre-processing

The first pre-processing step we apply is converting all dates into a more standardized format.

In [ ]:
%%time

from datetime import datetime

format_str = '%a %b %d %H:%M:%S %z %Y' #see https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior
def convert_date(date_string):
  return pd.to_datetime(date_string, utc=True, format=format_str)

_df_interactions = df_interactions.copy()
# _df_interactions['date_updated'] =  _df_interactions['date_updated'].progress_apply(convert_date)
_df_interactions['date_updated'] = _df_interactions['date_updated'].progress_apply(lambda s: np.datetime64(datetime.strptime(s,format_str)))
_df_interactions['date_updated'] = _df_interactions['date_updated'].dt.tz_localize(None)  # drops utc timezone

Now we define a pre-processing function that:

1. Drops ratings below 1, as we consider these to be non-relevant items fo rthe user.
2. Removes duplicate (user, item) pairs.
3. Removes users that occur in less than minsup interactions.

In [ ]:
def preprocess(df, minsup=10, min_score=1):
    """
    Goal: - Remove reconsumption items
          - Remove users that have less than minsup interactions 
          - Drop ratings == 0, i.e. "not provided"
               
    :input df: Dataframe containing user_id, item_id and time
    """
    # drop 0 ratings
    before = df.shape[0]
    df = df[(df["rating"] >= min_score)]
    print(f"After dropping ratings below {min_score}: {before} -> {df.shape[0]}")
    # drop reconsumption items
    before = df.shape[0]
    df = df.drop_duplicates(subset=["user_id","book_id"])
    print(f"After drop_duplicates (reconsumption items): {before} -> {df.shape[0]}")
    # drop users with less then minsup items in history
    g = df.groupby('user_id', as_index=False)['book_id'].size()
    g = g.rename({'size': 'user_sup'}, axis='columns')
    g = g[g.user_sup >= minsup]
    df = pd.merge(df, g, how='inner', on=['user_id'])
    print(f"After dropping users with less than {minsup} interactions: {before} -> {df.shape[0]}")
    return df

Then we apply the pre-processing function to the dataframe and log the change in number of samples, number of unique users and number of unique items.

In [ ]:
#print number of users and items
print(f"number of unique users: {_df_interactions['user_id'].nunique()}")
print(f"number of unique items: {_df_interactions['book_id'].nunique()}")
processed_df_interactions = preprocess(_df_interactions.copy())
# display(processed_df_interactions.head(5))
print(f"number of unique users: {processed_df_interactions['user_id'].nunique()}")
print(f"number of unique items: {processed_df_interactions['book_id'].nunique()}")
# create sequential ids
processed_df_interactions['user_id_seq'] = processed_df_interactions['user_id'].astype('category').cat.codes
processed_df_interactions['book_id_seq'] = processed_df_interactions['book_id'].astype('category').cat.codes
# merge book id and rating for easier 
display(processed_df_interactions.head(5))

We store the mapping from user/book id's to their sequential id in an external file. This might come in handy in other notebooks.

In [ ]:
processed_df_interactions[['user_id', 'user_id_seq']].drop_duplicates().to_pickle("./data/user_id_map.pkl")
processed_df_interactions[['book_id', 'book_id_seq']].drop_duplicates().to_pickle("./data/book_id_map.pkl")

We sort the interactions by their date and group them by user. This allows us to perform a session-based train-test split.

In [ ]:
# Sort on date and group per user
sessions_df = processed_df_interactions.sort_values(['date_updated'],ascending=True).groupby(by='user_id_seq', as_index=False)[['book_id_seq','date_updated', 'rating']].agg(list)

Perform session-based split.

In [ ]:
# Sort on date and group per user
sessions_df = processed_df_interactions.sort_values(['date_updated'],ascending=True).groupby(by='user_id_seq', as_index=False)[['book_id_seq', 'rating', 'date_updated']].agg(list)

# Function to perform split
def split(row, col, percentage_train):
    items = row[col]
    no_train_items = math.floor(len(items) * percentage_train)
    return items[0:no_train_items], items[no_train_items:]

# Split dataset into 0.7 training and 0.3 test samples, split in the temporal dimension.
percentage_train = 0.7
# train_items, test_items = split(items, percentage_train)
sessions_df[['history', 'future']] = sessions_df.progress_apply(lambda row: split(row, 'book_id_seq', percentage_train), axis=1, result_type='expand')
sessions_df[['history_ratings', 'future_ratings']] = sessions_df.progress_apply(lambda row: split(row, 'rating', percentage_train), axis=1, result_type='expand')
display(sessions_df.head(5))

Finally, we create a sparse representation of the user-item interaction matrix for our train and test set.

In [ ]:
def create_sparse_repr(df, column, shape):
    user_ids = []
    item_ids = []
    values = []
    for idx, row in tqdm(df.iterrows()):
        items = row[column]
        item_ids.extend(items)
        user = row['user_id_seq']
        user_ids.extend([user] * len(items))
        ratings = row[column + "_ratings"]
        values.extend(ratings)
    matrix = scipy.sparse.coo_matrix((values, (user_ids, item_ids)), shape=shape, dtype=np.int32)
    return matrix
    

shape = (processed_df_interactions['user_id_seq'].max() + 1,  processed_df_interactions['book_id_seq'].max() + 1)
train = create_sparse_repr(sessions_df, column='history', shape=shape)
test = create_sparse_repr(sessions_df, column='future', shape=shape)

We store the train and test set externally to be used in the training and evaluating notebook.

In [ ]:
scipy.sparse.save_npz('./data/train.npz', train)
scipy.sparse.save_npz('./data/test.npz', test)